In [ ]:
!pip install streamlit roboflow supervision opencv-python-headless pyngrok


In [ ]:
code = '''
import streamlit as st
from roboflow import Roboflow
import supervision as sv
import numpy as np
import cv2
import tempfile
import os

st.set_page_config(page_title="Smart Waste Detection", layout="centered")
st.title("♻️ Smart Waste Detection App")
st.write("Upload an image to detect waste objects using Roboflow.")

uploaded_file = st.file_uploader("📤 Upload Image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        temp_file.write(uploaded_file.read())
        img_path = temp_file.name

    st.image(img_path, caption="Uploaded Image", use_column_width=True)

    rf = Roboflow(api_key="GofFNfqAyIEcG9LAlOV8")
    model = rf.workspace().project("smart-waste-management-h5yif-mwcpw").version(1).model

    with st.spinner("🔍 Detecting..."):
        results = model.predict(img_path, confidence=40, overlap=30).json()
        predictions = results["predictions"]

    xyxy, confidence, labels, class_ids = [], [], [], []

    for pred in predictions:
        w, h = pred["width"], pred["height"]
        x, y = pred["x"], pred["y"]
        x1, y1 = x - w / 2, y - h / 2
        x2, y2 = x + w / 2, y + h / 2

        xyxy.append([x1, y1, x2, y2])
        confidence.append(pred["confidence"])
        labels.append(f"{pred['class']} ({pred['confidence']:.2f})")
        class_ids.append(0)

    detections = sv.Detections(
        xyxy=np.array(xyxy),
        confidence=np.array(confidence),
        class_id=np.array(class_ids)
    )

    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    box_annotator = sv.BoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    annotated = box_annotator.annotate(scene=image_rgb.copy(), detections=detections)
    annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

    st.image(annotated, caption="🧾 Detected Waste", use_column_width=True)

    output_path = os.path.join(tempfile.gettempdir(), "output.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
    with open(output_path, "rb") as file:
        st.download_button("⬇️ Download Annotated Image", file, "output.jpg", mime="image/jpeg")

    st.markdown("### 📊 Detected Classes")
    if predictions:
        detected_classes = set(pred["class"] for pred in predictions)
        for cls in detected_classes:
            st.markdown(f"- ✅ **{cls}**")
    else:
        st.warning("No objects detected.")
'''

with open("app.py", "w") as f:
    f.write(code)


In [ ]:
from pyngrok import ngrok
import os

# Kill any existing tunnels
ngrok.kill()

# Start Streamlit (runs in background)
!streamlit run app.py &> log.txt &

# Create HTTP tunnel on port 8501
public_url = ngrok.connect(8501, "http")
print("🌐 Streamlit app is live at:", public_url)



In [ ]:
!ngrok config add-authtoken 30fgGysZyTraDtUvOiuEaPH2IuK_35HiuGsPtqdCewCpNZTsP